In [ ]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
%matplotlib inline

#Turn off scientific notation in Pandas
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Import data
tn_budget = pd.read_csv('Data/Zipped_Data/tn.movie_budgets.csv.gz', compression = 'gzip')
bom_gross = pd.read_csv('Data/Zipped_Data/bom.movie_gross.csv.gz', compression = 'gzip')
imbd_basics = pd.read_csv('Data/Zipped_Data/imdb.title.basics.csv.gz', compression = 'gzip')
imbd_name = pd.read_csv('Data/Zipped_Data/imdb.name.basics.csv.gz', compression = 'gzip')
imbd_principals = pd.read_csv('Data/Zipped_Data/imdb.title.principals.csv.gz', compression = 'gzip')

In [ ]:
#Remove punctuation from producton and worldwide. Switch variables from string to float.
tn_budget['production_budget'] = tn_budget['production_budget'].apply(lambda x: x.translate(str.maketrans('', '', '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')))
tn_budget['worldwide_gross'] = tn_budget['worldwide_gross'].apply(lambda x: x.translate(str.maketrans('', '', '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')))
tn_budget['production_budget'] = pd.to_numeric(tn_budget.production_budget, downcast = 'float', errors = 'coerce')
tn_budget['worldwide_gross'] = pd.to_numeric(tn_budget.worldwide_gross, downcast = 'float', errors = 'coerce')

#Create column for net profit (worldwide gross minus production budget).
tn_budget['net'] = list(tn_budget.apply(lambda x: x.worldwide_gross - x.production_budget, axis=1))

#Create column for ratio (net profit divided by production budget)
tn_budget['ratio'] = list(tn_budget.apply(lambda x: x.net / x.production_budget, axis=1))

# Rename column 'movie' to 'title'
tn_budget = tn_budget.rename(columns = {'movie': 'title'})

#remove whitespace, potential extra words, punctuation, and case from titles
years = ['\(2010\)', "\(2011\)", 
         "\(2012\)", "\(2013\)", "\(2014\)", 
         "\(2015\)", "\(2016\)", "\(2017\)", "\(2018\)"]
tn_budget.title = tn_budget.title.replace(years, value='', regex=True)
tn_budget.title = tn_budget.title.str.strip()
tn_budget.title = tn_budget.title.apply(lambda x: x.lower())
tn_budget.title = tn_budget.title.apply(lambda x: x.translate(str.maketrans('', '', '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')))
tn_budget.title = tn_budget.title.replace(['the', 'and'], value='', regex=True)

#Remove month and year from date. Drop redundant columns
tn_budget['year'] = list(tn_budget['release_date'].str[-4:])
tn_budget = tn_budget.drop(columns = ['id', 'domestic_gross', 'release_date'])

In [ ]:
#Remove 'studio', 'domestic_gross', 'foreign_gross' column. Set 'foreign_gross' to float and 'year' to string
bom_gross = bom_gross.drop(columns = ['domestic_gross', 'foreign_gross'])
bom_gross['year'] = bom_gross['year'].astype(str)

#Remove years and right whitespace from titles. Remove potentially extraneous words.
years = ['\(2010\)', "\(2011\)", 
         "\(2012\)", "\(2013\)", "\(2014\)", 
         "\(2015\)", "\(2016\)", "\(2017\)", "\(2018\)"]
bom_gross.title = bom_gross.title.replace(years, value='', regex=True)
bom_gross.title = bom_gross.title.str.strip()
bom_gross.title = bom_gross.title.apply(lambda x: x.lower())
bom_gross.title = bom_gross.title.apply(lambda x: x.translate(str.maketrans('', '', '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')))
bom_gross.title = bom_gross.title.replace(['the', 'and'], value='', regex=True)

In [ ]:
#Merge tn_budget and bom_gross
merged_df = pd.merge(tn_budget, bom_gross, on = ["title", "year"], how = "left")

#Replace null studio values
merged_df['studio'] = merged_df.studio.fillna('Unknown')

In [ ]:
#Create 'year' string column from 'start_year'
imbd_basics['year'] = imbd_basics['start_year'].astype(str)

#Drop unused columns
imbd_basics = imbd_basics.drop(columns = ['original_title', 'start_year'])

#Rename column for easier merging
imbd_basics = imbd_basics.rename(columns = {'primary_title': 'title'})

#Remove years and right whitespace from titles. Remove potentially extraneous words.
years = ['\(2010\)', "\(2011\)", 
         "\(2012\)", "\(2013\)", "\(2014\)", 
         "\(2015\)", "\(2016\)", "\(2017\)", "\(2018\)"]
imbd_basics.title = imbd_basics.title.replace(years, value='', regex=True)
imbd_basics.title = imbd_basics.title.str.strip()
imbd_basics.title = imbd_basics.title.apply(lambda x: x.lower())
imbd_basics.title = imbd_basics.title.apply(lambda x: x.translate(str.maketrans('', '', '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')))
imbd_basics.title = imbd_basics.title.replace(['the', 'and'], value='', regex=True)

In [ ]:
#Merge imbd_basics with merged datafile
merged_df2 = pd.merge(merged_df, imbd_basics, on = ["title", "year"], how = "left")

In [ ]:
#Change year to integer, and remove all films made before 2010. Turn year back to string.
merged_df2['year'] = merged_df2['year'].astype(int)
merged_df2 = merged_df2.loc[merged_df2.year >= 2010]
merged_df2['year'] = merged_df2['year'].astype(str)

In [ ]:
#Merge imbd_principals and imbd_name
imbd_name_prin = pd.merge(imbd_principals, imbd_name, on = ["nconst"], how = "left")

In [ ]:
#Remove extraneous columns
imbd_name_prin = imbd_name_prin.drop(columns = ['ordering', 'nconst', 
                               'job', 'characters', 'birth_year', 'death_year', 
                               'primary_profession', 'known_for_titles'])

In [10]:
#Create list of directors from imbd_name_prin
directors = imbd_name_prin.loc[imbd_name_prin.category == 'director']

In [11]:
#Merge list of directors with merged document
merged_df3 = pd.merge(merged_df2, directors, on = ["tconst"], how = "left")

In [12]:
#Remove extra columns, rename primary_name to director
merged_df3 = merged_df3.drop(columns = ['category', 'runtime_minutes', 'genres'])
merged_df3 = merged_df3.rename(columns = {'primary_name': 'director'})

In [13]:
#Fill in null values for director
merged_df3['director'] = merged_df3.director.fillna('Not listed')

In [34]:
merged_df3.head()

,title,production_budget,worldwide_gross,net,ratio,year,studio,tconst,runtime_minutes,genres,director
0,pirates of caribbean on stranger tides,410600000.00,1045663872.00,635063872.00,1.55,2011,BV,tt1298650,136.00,"Action,Adventure,Fantasy",Rob Marshall
1,dark phoenix,350000000.00,149762352.00,-200237648.00,-0.57,2019,Unknown,tt6565702,113.00,"Action,Adventure,Sci-Fi",Simon Kinberg
2,avengers age of ultron,330600000.00,1403014016.00,1072414016.00,3.24,2015,BV,tt2395427,141.00,"Action,Adventure,Sci-Fi",Joss Whedon
3,star wars ep viii last jedi,317000000.00,1316721792.00,999721792.00,3.15,2017,Unknown,NaN,nan,NaN,Not listed
4,star wars ep vii force awakens,306000000.00,2053311232.00,1747311232.00,5.71,2015,Unknown,NaN,nan,NaN,Not listed


In [14]:
#Create list of producers from imbd_name_prin
producers = imbd_name_prin.loc[imbd_name_prin.category == 'producer']

In [15]:
#Rename column primary_name to producer
producers = producers.rename(columns = {'primary_name': 'producer'})

In [16]:
#Drop column 'category'
producers = producers.drop(columns = 'category')

In [17]:
#Merge list of producers with merged document
merged_df4 = pd.merge(merged_df3, producers, on = ["tconst"], how = "left")

In [18]:
#Fill null producer values
merged_df4['producer'] = merged_df4.producer.fillna('Not listed')

In [19]:
#Remove duplicate titles
merged_df_cleaned = merged_df4.loc[merged_df4.title.duplicated() == False]

In [ ]:
#Drop extra column
merged_df_cleaned = merged_df_cleaned.drop(columns = 'tconst')

In [33]:
merged_df5.head()

,title,production_budget,worldwide_gross,net,ratio,year,studio,tconst,runtime_minutes,genres,director,producer
0,pirates of caribbean on stranger tides,410600000.00,1045663872.00,635063872.00,1.55,2011,BV,tt1298650,136.00,"Action,Adventure,Fantasy",Rob Marshall,Not listed
1,dark phoenix,350000000.00,149762352.00,-200237648.00,-0.57,2019,Unknown,tt6565702,113.00,"Action,Adventure,Sci-Fi",Simon Kinberg,Not listed
2,avengers age of ultron,330600000.00,1403014016.00,1072414016.00,3.24,2015,BV,tt2395427,141.00,"Action,Adventure,Sci-Fi",Joss Whedon,Kevin Feige
3,star wars ep viii last jedi,317000000.00,1316721792.00,999721792.00,3.15,2017,Unknown,NaN,nan,NaN,Not listed,Not listed
4,star wars ep vii force awakens,306000000.00,2053311232.00,1747311232.00,5.71,2015,Unknown,NaN,nan,NaN,Not listed,Not listed


In [20]:
##### Create a list of 20 studios with the top ratio means
studio_ratio_mean = merged_df_cleaned.groupby(['studio']).ratio.mean().sort_values(ascending=False)[:20]

#Create a list of 20 directors with the top ratio means
director_ratio_mean = merged_df_cleaned.groupby(['director']).ratio.mean().sort_values(ascending=False)[:20]

#Create a list of 20 producers with the top ratio means
producer_ratio_mean = merged_df_cleaned.groupby(['producer']).ratio.mean().sort_values(ascending=False)[:20]

In [36]:
studio_ratio_mean

studio
WB (NL)    14.79
UTV        10.13
FD          8.94
Orch.       8.54
GrtIndia    7.78
BH Tilt     6.90
Uni.        5.45
Par.        5.17
MBox        4.88
TriS        4.74
A24         4.71
ParV        4.48
PNT         4.28
Eros        3.99
FoxS        3.46
Wein.       3.39
LGF         3.24
Affirm      3.04
LG/S        3.04
W/Dim.      2.98
Name: ratio, dtype: float64

In [37]:
director_ratio_mean

director
Travis Cluff         415.56
Tod Williams          58.17
William Brent Bell    53.29
Bradley Parker        41.41
Jordan Peele          30.89
Nitesh Tiwari         30.02
Josh Boone            24.60
Alex Kendrick         23.66
Henry Joost           23.50
Barry Jenkins         21.58
Dan Trachtenberg      20.66
John R. Leonetti      19.74
Stiles White          19.66
Daniel Stamm          18.50
James DeMonaco        17.22
David F. Sandberg     17.13
Adam Robitel          15.79
Stephen Susco         15.43
James Wan             14.86
Drake Doremus         13.91
Name: ratio, dtype: float64

In [38]:
producer_ratio_mean

producer
Dean Schnider            415.56
Morris Paulson           100.76
Brian Witten              41.41
Lawrence Grey             28.76
Adam Donaghey             26.70
Malek Akkad               24.49
Mike Medavoy              24.48
Steven Schneider          21.65
Lindsey Weber             20.66
Seth Grahame-Smith        18.93
Brandt Andersen           18.59
Sébastien K. Lemercier    17.22
Jamie Patricof            15.57
Jim Beach                 15.27
Dede Gardner              15.23
Jason Blum                14.52
Teddy Schwarzman          14.18
Jonathan Schwartz         13.91
Siddharth Roy Kapur       13.78
Peter Safran              13.40
Name: ratio, dtype: float64